## Adquisición con osciloscopio Tektronix TDS1052B

En este tutorial, mostramos cómo utilizar la biblioteca de Python [**PyVISA**](https://pyvisa.readthedocs.io/en/latest/) para comunicarnos con el osciloscopio [**TDS1052B**](https://drive.google.com/file/d/1RWL1YynB5UbAedhVg5HxluuabhzyV4i3/view?usp=sharing) a través del estándar [**NI-VISA**](https://en.wikipedia.org/wiki/Virtual_instrument_software_architecture) y adquirir datos de la señal obtenida como resultado del proceso de medición en nuestro experimento.

En primer lugar, importamos las bibliotecas que utilizaremos en este script:

In [ ]:
import pyvisa                               # para comunicarnos con los equipos
import numpy as np                          # para operar con listas de datos
import pandas as pd                         # para manejar archivos de datos: Guardar, abrir, etc
import matplotlib.pyplot as plt             # para graficar 
import time                                 # para realizar pausas entre mediciones

Vamos a mostrar cómo efectuar dos tipos de adquisiciones distintas:

1. Adquisición de una curva mostrada por el osciloscopio.
2. Mediciones matemáticas sobre la señal.

Para ambos casos, en primer lugar, necesitamos definir un objeto que nos permita comunicarnos e interactuar con los instrumentos que tenemos conectados vía puerto USB, GPIB, Serial (RS232), Ethernet, etc. Este objeto se conoce como Resource Manager y lo invocamos desde la librería pyvisa:

In [ ]:
rm = pyvisa.ResourceManager()            # Defino objeto resource manager de pyvisa

Este se utiliza para: (a) detectar dispositivos conectados y listar todos los recursos disponibles y (b) crear interfases para comunicarnos con dispositivos.

Una vez defido el objeto **rm**, podemos listar los instrumentos conectados a nuestra PC: 

In [ ]:
instrumentos_conectados = rm.list_resources()           # lista con los instrumentos conectados

print (instrumentos_conectados)

De la lista obtenida en la celda anterior, vemos que cada instrumento es identificado con un *string* de la siguiente manera: `puerto::marca::modelo::NroSerie`. Para identificar cual de todos es el osciloscopio, buscamos el *modelo* en la etiqueta ubicada en la parte trasera del equipo. Una vez identificado, copiamos el nombre y lo pegamos en la variable *instrumento*, luego procedemos a establecer la comunicación con el equipo:

In [ ]:
instrumento = 'puerto::marca::modelo::NroSerie'         # colocar acá el instrumento asociado al oscilascopio 

osci = rm.open_resource(instrumento)                    # Establesco comunicación con el osciloscopio

El objeto *osci* será nuestra interfase, éste nos permite interactuar con el osciloscopio utilizando rutinas estándar. Generalmente, realizamos dos tipos de operaciones:

- Enviar un comando al instrumento para configurar parámetros o iniciar procesos sin esperar ninguna respuesta. Esto se realiza mediante rutinas de tipo **write**.
  
- Enviar un comando al instrumento y esperar una respuesta, como por ejemplo, obtener los datos que estoy midiendo o consultar el estado del instrumento. Esto se realiza mediante rutinas de tipo **query**.

El lenguaje que utilizamos para comunicarnos con el equipo está definido en el estándar [**SCPI**](https://en.wikipedia.org/wiki/Standard_Commands_for_Programmable_Instruments) (Standard Commands for Programmable Instruments), el cual es ampliamente utilizado para la comunicación remota con equipos de instrumentación como osciloscopios, multímetros y generadores de funciones.

### 1. Adquicisión de una curva mostrada por el osciloscopio

Estás utilizando el osciloscopio para medir la caída de tensión en un componente de tu circuito y visualizas una curva. Te interesa adquirir la curva observada para realizar algún tipo de análisis de datos en la PC. A continuación, te explico cómo hacerlo.

Primero, configuramos el equipo en AUTOSET. Esto le indica al osciloscopio que ajuste su configuración para optimizar la visualización de la señal que está midiendo, en particular nos interesa que realize un ajuste automático de la escala. Para esto, escribimos:

In [ ]:
osci.write('AUTOS EXEC')   # Equivalente a apretar el boton AUTOSET del equipo, 
time.sleep(5)                   

La rutina *time.sleep()* genera una pausa de 5 segundos, tiempo de espera necesario para que el equipo optimice la visualización de la señal antes de poder recibir otra orden.

Una vez transcurrido el tiempo de espera, encendemos el canal del equipo con el que queremos adquirir, en este caso, el canal 1 (CH1):

In [ ]:
osci.write('SEL:CH1 ON')        # select:channel1 ON

Ahora le indicamos al equipo cómo debe codificar los datos de salida. En este caso, utilizaremos datos **Raw Binary** (RPB), lo que significa que queremos los datos en formato binario crudo, sin ningún tipo de procesamiento o compresión:

In [ ]:
osci.write('DAT:ENC RPB')

A continuación, seteamos el ancho de banda en *1 byte*. Es decir, establecemos que el osciloscopio envie datos como valores de 8 bits por muestra:

In [ ]:
osci.write('DAT:WID 1')         # data band width 1 byte

Configuramos el osciloscopio para que la fuente de datos sea el canal 1 (CH1). Esto significa que, al realizar operaciones de adquisición o cálculos posteriormente, el osciloscopio usará la señal medida en el canal 1:

In [ ]:
osci.write('DAT:SOU CH1')       # data:source channel1

Si estamos por tomar los datos y la pantalla se mueve mucho, por ejemplo por una señal ruidosa, es conveniente congelar la pantalla antes de medir. Eso se hace con el comando:

In [ ]:
osci.write('ACQ:STATE OFF')    

Pedimos también que los datos sean transmitidos limpiamente, sin encabezados:

In [ ]:
osci.write('HEADER OFF')

Una vez fijada la fuente de datos y congelado la pantalla, solicitamos al osciloscopio algunos parámetros relacionados con la escala de la señal que estamos adquiriendo:

In [ ]:
xze, xin, yze, ymu, yoff = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';') #waveform preamble information

En este comando multiple estamos solicitando: 

- 'WFMPRE:XZE?': Pide el desplazamiento horizontal inicial (XZE), que es el tiempo correspondiente al primer punto de la señal.
- 'WFMPRE:XIN?': Pide el intervalo de muestreo (XIN), que es el tiempo entre muestras consecutivas.
- 'WFMPRE:YZE?': Pide el desplazamiento vertical cero (YZE), que es el nivel de voltaje que corresponde a un valor digital de cero.
- 'WFMPRE:YMU?': Pide la escala vertical (YMU), que convierte unidades digitales a unidades físicas (típicamente voltios).
- 'WFMPRE:YOFF?': Pide el desplazamiento vertical (YOFF), que corrige la señal para que el cero físico corresponda al cero digital.

Más adelante utilizaremos estos parámetros para escribir la curva en terminos de las magnitudes físicas correspondientes.

Ahora si, finalmente, adquirimos la curva que observamos en el osciloscopio:

In [ ]:
dataCH1 = osci.query_binary_values('CURV?', datatype='B', container=np.array) 

Los argumentos de la función *osci.query_binary_values()* indican:

- 'CURV?': Es la consulta que se realiza al dispositivo. En este caso, 'CURV?' generalmente se usa para solicitar datos de forma de onda (curva) del osciloscopio. Esto indica que el dispositivo debe enviar datos binarios de la forma de onda.

- datatype='B': Especifica el tipo de datos que se esperan recibir del dispositivo. 'B' indica que los datos se recibirán como bytes (es decir, valores enteros de 8 bits sin signo).

- container=np.array: Especifica el contenedor en el que se deben almacenar los datos recibidos. *np.array* indica que los datos se almacenarán en un array de NumPy.

Hecha la medición, podemos descongelar la pantalla para eventualmente realizar otra medición:

In [ ]:
osci.write('ACQ:STATE ON')

Ya con la curva adquirida, utilizamos los parámetros para re-escribir la curva en terminos de las magnitudes físicas correspondientes: 

In [ ]:
t = xze + np.arange(len(dataCH1)) * xin             # Conversion a tiempo en segundos

VCH1 = (dataCH1 - yoff) * ymu + yze                 # Conversion a voltaje en Volts

Hacemos un gráfico para visualizar los datos:

In [ ]:
fig, ax  =  plt.subplots()

ax.plot(t, VCH1, label="CH1")

ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Voltaje [V]')

plt.show()

Queremos guardar los datos medidos, una forma sencilla es hacerlo a través de la libreria *pandas*. Primero definimos un objeto *DataFrame* al cual le pasamos como argumentos los datos y los nombres de las columnas:

In [ ]:
df = pd.DataFrame( [t, VCH1], columns=["tiempo", "voltaje"] )  

El objeto resultante, *df*, es un tipo de estructura de datos en Pandas que funciona como una tabla, similar a una hoja de cálculo o una base de datos, donde a partir de varias rutinas de la librería, podemos almacenar y manipular datos organizados en filas y columnas.

Hecho esto, definimos una carpeta de destino para los datos, un nombre, y guardamos el conjunto de datos en un archivo tipo *Comma Separated Values* (*.csv*):

In [ ]:
carpeta = "path/to/you/folder/"                       # indicar direccion de la carpeta donde se guardarán los datos

df.to_csv( carpeta+"voltaje_CH1.csv", index=False )

### 2. Mediciones matemáticas sobre la señal

Estamos midiendo la caída de tensión sobre un componente de nuestro circuito. Para esto estamos utilizando el canal 1 del osciloscopio (CH1). Visualizamos la curva en el osciloscopio y queremos obtener de ésta la amplitud y la fase. Para realizar estas mediciones, vamos a utilizar el comando **MEASUREMENT**.

#### 2.1. Medición de amplitud de la señal

En primer lugar configuramos al canal 1 como la fuente para la medición número 1 en el osciloscopio:

In [ ]:
osci.write('MEASU:MEAS1:SOURCE CH1')

Para medir la amplitud, configuramos al osciloscopio para que la medición 1 sea del tipo *Root Mean Square* (RMS), es decir pedimos que cuando midamos nos entregue el voltaje eficaz de la señal:

In [ ]:
osci.write('MEASU:MEAS1:TYPE CRMS')  

Antes de medir ajusto la escala del ociloscopio y le doy 5 segundos de pausa antes de ejecutar un nuevo comando

In [ ]:
osci.write('AUTOS EXEC')
time.sleep(5)

Mido la amplitud en el canal 1:

In [ ]:
ACH1= float(osci.query('MEASU:MEAS1:VALue?')) # 

Note que *osci.query('MEASU:MEAS1:VALue?')* devuelve un string con el valor medido, como por ejemplo "0.1234" o "1.56E-03". Para convertirlo a un número flotante y poder hacer cálculos con él, es necesario usar la rutina *float()*.

Por último queremos obtener la incertidumbre en la medicion, para eso utilizamos la expresión:


$\Delta A = \Delta A_{\text{rel}} + \Delta A_{\text{res}}$

Donde:  
- $ \Delta A_{\text{rel}}$: Error relativo asociado al sistema de medición.  
- $ \Delta A_{\text{res}}$: Error debido a la resolución del sistema (cuantización).

Cálculo de cada término:

1. **Error relativo ($ \Delta A_{\text{rel}}$)**  
    $\Delta A_{\text{rel}} = \alpha ACH1$
    
    Donde:  
    - $ACH1$ es la amplitud medida.
    - $\alpha$ es el error relativo dado por el fabricante del dispositivo (en este osciloscopio, $\alpha = 0.03$ o $3\%$).

2. **Error de resolución ($\Delta A_{\text{res}}$)**  
   $\Delta A_{\text{res}} = \frac{\text{escalav1} \times (R/N)}{2}$

   Donde:  
   - $\text{escalav1}$ es la escala de medición (por ejemplo 0.1 V/div).  
   - $R$ es el rango total del sistema, es la diferencia entre el máximo y el mínimo voltaje que el osciloscopio puede medir.
    En este modelo R=10 V (mide de -5 V a +5 V).  
   - $N$ es el número de niveles de cuantización, como estoy midiendo con 8 bits, N = 255.
   - El factor 2 indica estamos considerando que el error de resolución podría ser de hasta la mitad del intervalo de cuantización.

Note que, necesitamos la escala vertical (o sensibilidad) del canal 1 para realizar el cálculo. Para obtenerla ejecutamos:

In [ ]:
escalav1 = float(osci.query('CH1:SCAle?'))

El error de medición entonces se calcula como:

In [ ]:
errACH1 = 0.03*ACH1 + (escalav1*(10/255))/2.0

#### 2.2. Medición de la fase de la señal

Al igual que antes utilizamos el comando MEASUREMENT, el procedimiento es similar al caso de la amplitud: 

Definimos la medicion 2 (MEAS2) con fuente en el canal 1, configuramos el tipo de medicion 2 en *tipo fase*, y ejecutamos un *query* para adquirir el valor:

In [ ]:
osci.write('MEASU:MEAS2:SOURCE CH1')

osci.write('MEASU:MEAS2:TYPE PHAse')

fase = osci.query_ascii_values('MEASU:MEAS3:VAL?')      # en radianes (tipo float)

Nota: ¿Cómo mide la fase el osciloscopio?

- Calcula el instante en el que la señal cruza el eje horizontal en la dirección positiva por primera vez (cero cruzado ascendente de la señal ), $\Delta t$.
- Calcula el período de la señal, T.
- Para cálcular la fase utiliza la relación $\phi = 2 \pi \frac{\Delta t}{T}$.